In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from tqdm import tqdm

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    certifi-2019.3.9           |           py36_0         149 KB  conda-forge
    conda-4.6.8                |           py36_0         876 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    openssl-1.1.1b             |       h14c3975_1         4.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0

The follow

In [2]:
# import json already did 
from pprint import pprint
with open('nyu_2451_34572-geojson.json') as f:
    newyork_data = json.load(f)



In [3]:
neighbor_data = newyork_data['features']

In [4]:

neighbor_data[1]

{'type': 'Feature',
 'id': 'nyu_2451_34572.2',
 'geometry': {'type': 'Point',
  'coordinates': [-73.82993910812398, 40.87429419303012]},
 'geometry_name': 'geom',
 'properties': {'name': 'Co-op City',
  'stacked': 2,
  'annoline1': 'Co-op',
  'annoline2': 'City',
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.82993910812398,
   40.87429419303012,
   -73.82993910812398,
   40.87429419303012]}}

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
for data in neighbor_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [8]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [9]:

print('The Brooklyn borough has {} neighborhoods.'.format(
        brooklyn_data.shape[0]
    )
)

The Brooklyn borough has 70 neighborhoods.


In [10]:
address = 'Brooklyn, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


In [11]:
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

In [12]:
# add markers to map
for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

In [13]:
CLIENT_ID = '5U2VS5YVDH0G1VS3ZFBMWT35M42PRLDJOPOMMBPKAZLHO4ZS' # your Foursquare ID
CLIENT_SECRET = 'YUG3DWDF1WYFAAFQ2IE3I0N5JMLUBIUACJ2Q4JQXQQTT4OFX' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=200):
    
    pbar = tqdm(total=36)
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):        
        pbar.update(1)
        
 

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

        
    pbar.close()
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

print('Brooklyn has {} Neighborhood and total {} venues.'.format(
        len(brooklyn_venues['Neighborhood'].unique()),
        brooklyn_venues.shape[0]
    )
)

70it [00:24,  4.10it/s]


Brooklyn has 70 Neighborhood and total 2810 venues.


In [16]:
brooklyn_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
4,Bay Ridge,40.625801,-74.030621,Ho' Brah Taco Joint,40.622960,-74.031371,Taco Place


In [17]:
print('There are {} uniques categories.'.format(len(brooklyn_venues['Venue Category'].unique())))

There are 290 uniques categories.


In [18]:
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

In [19]:
brooklyn_onehot.head()

,Yoga Studio,Adult Boutique,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant

In [20]:
brooklyn_onehot.shape

brooklyn_restaurant = brooklyn_onehot.filter(regex='Neighborhood|Restaurant|Pizza|Coffee|Café|Food Court')

brooklyn_restaurant.shape

(2810, 62)

In [21]:
other_cols = [col for col in brooklyn_onehot.columns if 'Restaurant' not in col\
                                                     and 'Pizza' not in col\
                                                     and 'Coffee' not in col\
                                                     and 'Café' not in col\
                                                     and 'Food Court' not in col]

brooklyn_others = brooklyn_onehot[other_cols]
brooklyn_others.shape

(2810, 229)

In [22]:
brooklyn_restaurant_grouped = brooklyn_restaurant.groupby('Neighborhood').mean().reset_index()
brooklyn_restaurant_grouped.head()

,Neighborhood,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Coffee Shop,Cuban Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Food Court,French Restaurant,German Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Hotpot Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,North Indian Restaurant,Pakistani Restaurant,Peruvian Restaurant,Pizza Place,Polish Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Bath Beach,0.000000,0.0,0.0,0.020833,0.020833,0.0,0.020833,0.0,0.000000,0.062500,0.020833,0.0,0.000000,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,0.0,0.020833,0.000000,0.020833,0.0,0.000000,0.000000,0.0,0.041667,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.020833,0.041667,0.0,0.0,0.020833,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,0.000000,0.0,0.020833,0.0,0.000000
1,Bay Ridge,0.034091,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.011364,0.022727,0.000000,0.0,0.011364,0.0,0.0,0.0,0.0,0.011364,0.0,0.0,0.0,0.000000,0.045455,0.000000,0.0,0.000000,0.011364,0.0,0.068182,0.000000,0.0,0.0,0.0,0.0,0.0,0.011364,0.011364,0.022727,0.011364,0.0,0.0,0.000000,0.045455,0.0,0.0,0.000000,0.0,0.022727,0.000000,0.0,0.0,0.0,0.0,0.022727,0.0,0.0,0.022727,0.0,0.000000,0.0,0.011364
2,Bedford Stuyvesant,0.000000,0.0,0.0,0.000000,0.076923,0.0,0.000000,0.0,0.000000,0.000000,0.115385,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.038462,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.038462,0.0,0.0,0.000000,0.076923,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.038462
3,Bensonhurst,0.032258,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.096774,0.032258,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.032258,0.000000,0.0,0.032258,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.064516,0.0,0.0,0.000000,0.0,0.000000,0.032258,0.0,0.0,0.0,0.0,0.064516,0.0,0.0,0.000000,0.0,0.000000,0.0,0.032258
4,Bergen Beach,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [23]:

brooklyn_restaurant_grouped.shape

(70, 62)

In [24]:
brooklyn_others_grouped = brooklyn_others.groupby('Neighborhood').mean().reset_index()
brooklyn_others_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Airport Terminal,Antique Shop,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Candy Store,Cheese Shop,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Event Service,Event Space,Factory,Farm,Farmers Market,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Stand,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Indie Theater,Intersection,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Lake,Laser Tag,Laundry Service,Leather Goods Store,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Men's Store,Metro Station,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Racetrack,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Taco Place,Tattoo Parlor,Tea Room,Tennis Court,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trail,Used Bookstore,Vape Store,Varenyky restaurant,Video Game Store,Video Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Bath Beach,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.020833,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.041667,0.0,0.020833,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.020833,0.000000,0.0,0.0,0.0,0.020833,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.020833,0.020833,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,0.0,0.000000,0.0,0.020833,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.041667,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.020833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,0.0,0.020833,0.062500,0.0,0.0,0.0,0.0,0.020833,0.0,0.020833,0.0,0.000000,0.0,0.0,0.0,0.0,0.020833,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,

In [25]:
def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:

num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Restaurant'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Restaurant'.format(ind+1))

# create a new dataframe
neighborhoods_restaurant_sorted = pd.DataFrame(columns=columns)
neighborhoods_restaurant_sorted['Neighborhood'] = brooklyn_restaurant_grouped['Neighborhood']

for ind in np.arange(brooklyn_restaurant_grouped.shape[0]):
    neighborhoods_restaurant_sorted.iloc[ind, 1:] = most_common_venues(brooklyn_restaurant_grouped.iloc[ind, :], num_top_venues)

neighborhoods_restaurant_sorted.head()


,Neighborhood,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant,11th Most Common Restaurant,12th Most Common Restaurant,13th Most Common Restaurant,14th Most Common Restaurant,15th Most Common Restaurant
0,Bath Beach,Chinese Restaurant,Italian Restaurant,Sushi Restaurant,Pizza Place,Fast Food Restaurant,Asian Restaurant,Café,Halal Restaurant,Cantonese Restaurant,German Restaurant,Coffee Shop,Restaurant,Turkish Restaurant,Peruvian Restaurant,Hawaiian Restaurant
1,Bay Ridge,Italian Restaurant,Pizza Place,Greek Restaurant,American Restaurant,Chinese Restaurant,Thai Restaurant,Sushi Restaurant,Seafood Restaurant,Middle Eastern Restaurant,Caucasian Restaurant,Dim Sum Restaurant,Fast Food Restaurant,Indian Restaurant,Mediterranean Restaurant,Mexican Restaurant
2,Bedford Stuyvesant,Coffee Shop,Café,Pizza Place,Vietnamese Restaurant,New American Restaurant,Japanese Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Israeli Restaurant,Indian Restaurant,Hotpot Restaurant,Hawaiian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant
3,Bensonhurst,Chinese Restaurant,Pizza Place,Sushi Restaurant,Vietnamese Restaurant,Shabu-Shabu Restaurant,Coffee Shop,Hotpot Restaurant,Italian Restaurant,American Restaurant,Southern / Soul Food Restaurant,South American Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant
4,Bergen Beach,Vietnamese Restaurant,Japanese Restaurant,Israeli Restaurant,Indian Restaurant,Hotpot Restaurant,Hawaiian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Food Court,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant


In [27]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_others_sorted = pd.DataFrame(columns=columns)
neighborhoods_others_sorted['Neighborhood'] = brooklyn_others_grouped['Neighborhood']

for ind in np.arange(brooklyn_others_grouped.shape[0]):
    neighborhoods_others_sorted.iloc[ind, 1:] = most_common_venues(brooklyn_others_grouped.iloc[ind, :], num_top_venues)

neighborhoods_others_sorted.head()

neighborhoods_restaurant_sorted.head()

,Neighborhood,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant,11th Most Common Restaurant,12th Most Common Restaurant,13th Most Common Restaurant,14th Most Common Restaurant,15th Most Common Restaurant
0,Bath Beach,Chinese Restaurant,Italian Restaurant,Sushi Restaurant,Pizza Place,Fast Food Restaurant,Asian Restaurant,Café,Halal Restaurant,Cantonese Restaurant,German Restaurant,Coffee Shop,Restaurant,Turkish Restaurant,Peruvian Restaurant,Hawaiian Restaurant
1,Bay Ridge,Italian Restaurant,Pizza Place,Greek Restaurant,American Restaurant,Chinese Restaurant,Thai Restaurant,Sushi Restaurant,Seafood Restaurant,Middle Eastern Restaurant,Caucasian Restaurant,Dim Sum Restaurant,Fast Food Restaurant,Indian Restaurant,Mediterranean Restaurant,Mexican Restaurant
2,Bedford Stuyvesant,Coffee Shop,Café,Pizza Place,Vietnamese Restaurant,New American Restaurant,Japanese Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Israeli Restaurant,Indian Restaurant,Hotpot Restaurant,Hawaiian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant
3,Bensonhurst,Chinese Restaurant,Pizza Place,Sushi Restaurant,Vietnamese Restaurant,Shabu-Shabu Restaurant,Coffee Shop,Hotpot Restaurant,Italian Restaurant,American Restaurant,Southern / Soul Food Restaurant,South American Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant
4,Bergen Beach,Vietnamese Restaurant,Japanese Restaurant,Israeli Restaurant,Indian Restaurant,Hotpot Restaurant,Hawaiian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Food Court,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant


In [28]:
# set number of clusters
kclusters = 5

brooklyn_others_clustering = brooklyn_others_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_others_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([2, 2, 1, 1, 0, 1, 2, 2, 4, 1], dtype=int32)

In [29]:
brooklyn_others_merged = brooklyn_data

# add clustering labels
brooklyn_others_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
brooklyn_others_merged = brooklyn_others_merged.join(neighborhoods_others_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_others_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,2,Spa,Bagel Shop,Grocery Store,Bar,Sandwich Place,Playground,Pharmacy,Ice Cream Shop,Hookah Bar,Juice Bar,Optical Shop,Shoe Store,Bookstore,Breakfast Spot,Gym / Fitness Center
1,Brooklyn,Bensonhurst,40.611009,-73.995180,2,Ice Cream Shop,Bakery,Bagel Shop,Road,Liquor Store,Noodle House,Butcher,Spa,Grocery Store,Bar,Donut Shop,Supermarket,Cosmetics Shop,Smoke Shop,Park
2,Brooklyn,Sunset Park,40.645103,-74.010316,1,Bakery,Bank,Pharmacy,Mobile Phone Shop,Gym,Donut Shop,Record Shop,Sandwich Place,Shoe Store,Deli / Bodega,Supplement Shop,Grocery Store,Women's Store,Bagel Shop,Video Game Store
3,Brooklyn,Greenpoint,40.730201,-73.954241,1,Bar,Cocktail Bar,Yoga Studio,Boutique,Record Shop,Bakery,Grocery Store,Sandwich Place,Furniture / Home Store,Nightclub,Gastropub,Optical Shop,Liquor Store,Organic Grocery,Laundry Service
4,Brooklyn,Gravesend,40.595260,-73.973471,0,Bakery,Lounge,Bus Station,Martial Arts Dojo,Gourmet Shop,Spa,Music Venue,Grocery Store,Baseball Field,Bar,Gym,Pharmacy,Cosmetics Shop,Automotive Shop,Donut Shop


In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_others_merged['Latitude'],\
                                  brooklyn_others_merged['Longitude'],\
                                  brooklyn_others_merged['Neighborhood'],\
                                  brooklyn_others_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

In [31]:
map_clusters

In [32]:
print('FIRST CLUSTER')
brooklyn_others_merged.loc[brooklyn_others_merged['Cluster Labels'] == 0,\
                                brooklyn_others_merged.columns[[1]\
                              + list(range(5,brooklyn_others_merged.shape[1] -5))]]

FIRST CLUSTER


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Gravesend,Bakery,Lounge,Bus Station,Martial Arts Dojo,Gourmet Shop,Spa,Music Venue,Grocery Store,Baseball Field,Bar


In [33]:
print('SECOND CLUSTER')
brooklyn_others_merged.loc[brooklyn_others_merged['Cluster Labels'] == 1,\
                                brooklyn_others_merged.columns[[1]\
                              + list(range(5, brooklyn_others_merged.shape[1] -5))]]

SECOND CLUSTER


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Sunset Park,Bakery,Bank,Pharmacy,Mobile Phone Shop,Gym,Donut Shop,Record Shop,Sandwich Place,Shoe Store,Deli / Bodega
3,Greenpoint,Bar,Cocktail Bar,Yoga Studio,Boutique,Record Shop,Bakery,Grocery Store,Sandwich Place,Furniture / Home Store,Nightclub
5,Brighton Beach,Beach,Pharmacy,Mobile Phone Shop,Gourmet Shop,Bank,Bakery,Bookstore,Lounge,Non-Profit,Supermarket
9,Crown Heights,Museum,Playground,Bagel Shop,Pharmacy,Park,Candy Store,Salon / Barbershop,Burger Joint,Supermarket,Bakery
11,Kensington,Grocery Store,Sandwich Place,Deli / Bodega,Ice Cream Shop,Department Store,Donut Shop,Spa,Music Venue,Liquor Store,Lingerie Store
13,Prospect Heights,Bar,Cocktail Bar,Wine Shop,Bakery,Gym,Gourmet Shop,Beer Bar,Ice Cream Shop,Diner,Women's Store
15,Williamsburg,Bar,Bagel Shop,Yoga Studio,Park,Event Space,Clothing Store,Burger Joint,Breakfast Spot,Pet Store,Lounge
16,Bushwick,Deli / Bodega,Bar,Bakery,Discount Store,Thrift / Vintage Store,Diner,Shoe Store,Gym,Fruit & Vegetable Store,Sporting Goods Shop
21,Red Hook,Art Gallery,Park,Flower Shop,Wine Shop,Farm,Bagel Shop,Ice Cream Shop,Clothing Store,Gift Shop,Cocktail Bar
22,Gowanus,Bar,Furniture / Home Store,Gym / Fitness Center,Wine Shop,Rental Car Location,Art Gallery,Deli / Bodega,Yoga Studio,Brewery,Nightclub


In [34]:
print('THIRD CLUSTER')

brooklyn_others_merged.loc[brooklyn_others_merged['Cluster Labels'] == 2,\
                                brooklyn_others_merged.columns[[1]\
                              + list(range(5, brooklyn_others_merged.shape[1] -5))]]

THIRD CLUSTER


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Spa,Bagel Shop,Grocery Store,Bar,Sandwich Place,Playground,Pharmacy,Ice Cream Shop,Hookah Bar,Juice Bar
1,Bensonhurst,Ice Cream Shop,Bakery,Bagel Shop,Road,Liquor Store,Noodle House,Butcher,Spa,Grocery Store,Bar
6,Sheepshead Bay,Dessert Shop,Sandwich Place,Park,Hotel,Buffet,Creperie,Department Store,Diner,Grocery Store,Karaoke Bar
7,Manhattan Terrace,Grocery Store,Bagel Shop,Mobile Phone Shop,Bus Stop,Steakhouse,Organic Grocery,Convenience Store,Bank,Cosmetics Shop,Burger Joint
10,East Flatbush,Park,Wine Shop,Department Store,Pharmacy,Food,Print Shop,Supermarket,Garden Center,Bakery,Field
14,Brownsville,Moving Target,Park,Playground,Performing Arts Venue,Donut Shop,Farmers Market,Discount Store,Fried Chicken Joint,Pool,Pharmacy
17,Bedford Stuyvesant,Bus Stop,Bar,Boutique,BBQ Joint,Gourmet Shop,Cocktail Bar,Juice Bar,Thrift / Vintage Store,Bagel Shop,Fried Chicken Joint
18,Brooklyn Heights,Yoga Studio,Park,Deli / Bodega,Cosmetics Shop,Gym,Plaza,Pet Store,Ice Cream Shop,Gym / Fitness Center,Scenic Lookout
19,Cobble Hill,Bar,Yoga Studio,Cocktail Bar,Deli / Bodega,Ice Cream Shop,Playground,Boutique,Gift Shop,Bakery,Flower Shop
20,Carroll Gardens,Cocktail Bar,Bakery,Spa,Gym / Fitness Center,Bar,Wine Shop,Grocery Store,Dessert Shop,Food & Drink Shop,Wine Bar


In [35]:
print('FOURTH CLUSTER')
 
brooklyn_others_merged.loc[brooklyn_others_merged['Cluster Labels'] == 3,\
                                brooklyn_others_merged.columns[[1]\
                              + list(range(5, brooklyn_others_merged.shape[1] -5))]]

FOURTH CLUSTER


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,Prospect Park South,Pharmacy,Mobile Phone Shop,Grocery Store,Donut Shop,Clothing Store,Women's Store,Kids Store,Electronics Store,Bus Station,Sandwich Place


In [36]:
print('FIFTH CLUSTER')

brooklyn_others_merged.loc[brooklyn_others_merged['Cluster Labels'] == 4,\
                                brooklyn_others_merged.columns[[1]\
                              + list(range(5, brooklyn_others_merged.shape[1] -5))]]

FIFTH CLUSTER


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Flatbush,Deli / Bodega,Bus Station,Bank,Lounge,Pharmacy,Plaza,Diner,Sandwich Place,Donut Shop,Juice Bar
12,Windsor Terrace,Food Truck,Park,Diner,Plaza,Bagel Shop,Bar,Butcher,Thrift / Vintage Store,Bakery,Grocery Store
25,Cypress Hills,Fried Chicken Joint,Donut Shop,Ice Cream Shop,Men's Store,Bakery,Discount Store,Metro Station,Supermarket,Baseball Field,Dance Studio
28,Canarsie,Deli / Bodega,Gym,Bus Line,Food Truck,Food Stand,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market
38,Clinton Hill,Wine Shop,Yoga Studio,Arts & Crafts Store,Bagel Shop,Pet Store,Deli / Bodega,Boutique,Diner,Sandwich Place,Lounge
41,Boerum Hill,Dance Studio,Spa,Bar,Hotel,Arts & Crafts Store,Bakery,Cocktail Bar,Cosmetics Shop,Deli / Bodega,Furniture / Home Store
44,City Line,Donut Shop,Shoe Store,Fried Chicken Joint,Pharmacy,Food Truck,Mobile Phone Shop,Food,Burger Joint,Liquor Store,Ski Area
50,North Side,Burger Joint,Jewelry Store,Cocktail Bar,Bakery,Bar,Yoga Studio,Movie Theater,Juice Bar,Ice Cream Shop,Dive Bar
51,South Side,Bar,Yoga Studio,Burger Joint,Breakfast Spot,Wine Bar,Dive Bar,BBQ Joint,Nightclub,Movie Theater,Pet Store


In [ ]:
restaurant_type = input("restaurant type.. ")

In [ ]:
print('List the neighborhood which has the specific resturant in top 10 common venues')
neighborhoods_restaurant_sorted
col_num = neighborhoods_restaurant_sorted.shape[1]
# define the dataframe columns
col_names = ['Neighborhood', 'Type', 'Most Common'] 

# instantiate the dataframe
is_common_restaurant = pd.DataFrame(columns=col_names)

is_common_restaurant

In [ ]:
brooklyn_restaurant_filtered = is_common_restaurant

brooklyn_restaurant_filtered = brooklyn_restaurant_filtered.join(brooklyn_data.set_index('Neighborhood'), on='Neighborhood')

brooklyn_restaurant_filtered.drop(['Borough'], axis=1, inplace=True)
brooklyn_restaurant_filtered

In [ ]:
print('Map the Neighborhood where specific restaurant is common')
map_resturant = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
rainbow = colors = ['red', 'blue', 'gray', 'darkred', 'lightred', 'orange', 'beige',
    'green', 'darkgreen', 'lightgreen', 'darkblue', 'lightblue', 'purple', 'darkpurple',
    'pink', 'cadetblue', 'lightgray', 'black']

# add markers to the map
markers_colors = []
for lat, lon, poi, common in zip(brooklyn_restaurant_filtered['Latitude'],\
                                  brooklyn_restaurant_filtered['Longitude'],\
                                  brooklyn_restaurant_filtered['Neighborhood'],\
                                  brooklyn_restaurant_filtered['Most Common']):
    label = folium.Popup(str(poi) + ' Most Common: ' + str(common), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[common-1],
        fill=True,
        fill_color=rainbow[common-1],
        fill_opacity=0.7).add_to(map_resturant)
       
map_resturant

In [ ]:
print('Neighborhood where the specific restaurant is not in top 10')
brooklyn_restaurant_suitable = brooklyn_data

brooklyn_restaurant_suitable = brooklyn_restaurant_suitable.join(is_common_restaurant.set_index('Neighborhood'), on='Neighborhood')

brooklyn_restaurant_suitable.drop(['Borough'], axis=1, inplace=True)

brooklyn_restaurant_neighborhood_recommendation = brooklyn_restaurant_suitable.loc[brooklyn_restaurant_suitable['Most Common'].isnull()]
brooklyn_restaurant_neighborhood_recommendation.head()

In [ ]:
print('Final Results')
# create map
map_resturant = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
rainbow = colors = ['darkgreen','darkblue','purple', 'darkpurple',
    'pink', 'cadetblue', 'lightgray', 'black', 'red', 'blue', 'gray', 'darkred', 'lightred', 'orange', 'beige']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_restaurant_neighborhood_recommendation['Latitude'],\
                                  brooklyn_restaurant_neighborhood_recommendation['Longitude'],\
                                 brooklyn_restaurant_neighborhood_recommendation['Neighborhood'],\
                                  brooklyn_restaurant_neighborhood_recommendation['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_resturant)
       
map_resturant